In [1]:
import torch
import sys
sys.path.append('..')
from configs.culane.final_exp_res18_s8 import *
from resnet import ResNet
from collections import Iterable
# from lanepoints_conv import LanePointsConv
# from dcn_fpn import DeformFPN
# from mmdet.ops import DeformConv1D

/disk/zhangyunzhi/anaconda3/envs/ganet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/disk/zhangyunzhi/anaconda3/envs/ganet/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [5]:
from torch import nn
from torch import randn

In [68]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Conv2d(1, 1, 3, padding=1)
        self.head1 = nn.Conv2d(1, 1, 1)
        self.head2 = nn.Conv2d(1, 1, 1)
    def forward(self, x):
        x = self.backbone(x)
        return self.head1(x).sum().reshape(1), self.head2(x).sum().reshape(1)

dataset = [(randn(1,1,4,4), torch.Tensor([2])), (randn(1,1,4,4), torch.Tensor([3])), (randn(1,1,4,4), torch.Tensor([2]))]
net = Net()
sgd = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.L1Loss()
net.train()

Net(
  (backbone): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (head1): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (head2): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
)

In [69]:
origin_params = []
for params in net.parameters():
    origin_params.append(params)
origin_params

[Parameter containing:
 tensor([[[[-0.2794,  0.2380,  0.2667],
           [-0.2585,  0.2796, -0.0005],
           [ 0.2131,  0.1153, -0.1416]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.1325], requires_grad=True),
 Parameter containing:
 tensor([[[[-0.5082]]]], requires_grad=True),
 Parameter containing:
 tensor([0.1795], requires_grad=True),
 Parameter containing:
 tensor([[[[0.0334]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.6952], requires_grad=True)]

In [72]:
for epoch in range(10):
    for x, y in dataset:
        sgd.zero_grad()
        out = net(x)
        y = torch.Tensor(y)
        loss = abs(out[0] - y) + abs(out[1] - y)
        print(loss)
        loss.backward()
        sgd.step()
    after_params = []
for params in net.parameters():
    after_params.append(params)

tensor([2.0415], grad_fn=<AddBackward0>)
tensor([4.6729], grad_fn=<AddBackward0>)
tensor([5.6235], grad_fn=<AddBackward0>)
tensor([2.5016], grad_fn=<AddBackward0>)
tensor([2.5316], grad_fn=<AddBackward0>)
tensor([5.4308], grad_fn=<AddBackward0>)
tensor([3.6130], grad_fn=<AddBackward0>)
tensor([5.1263], grad_fn=<AddBackward0>)
tensor([3.9947], grad_fn=<AddBackward0>)
tensor([4.2285], grad_fn=<AddBackward0>)
tensor([2.1372], grad_fn=<AddBackward0>)
tensor([3.6323], grad_fn=<AddBackward0>)
tensor([1.8968], grad_fn=<AddBackward0>)
tensor([4.8007], grad_fn=<AddBackward0>)
tensor([5.7630], grad_fn=<AddBackward0>)
tensor([2.5840], grad_fn=<AddBackward0>)
tensor([2.4782], grad_fn=<AddBackward0>)
tensor([5.3370], grad_fn=<AddBackward0>)
tensor([3.5677], grad_fn=<AddBackward0>)
tensor([5.1639], grad_fn=<AddBackward0>)
tensor([4.0429], grad_fn=<AddBackward0>)
tensor([4.3072], grad_fn=<AddBackward0>)
tensor([2.1336], grad_fn=<AddBackward0>)
tensor([3.5829], grad_fn=<AddBackward0>)
tensor([1.8242],

In [73]:
[x==y for x, y in zip(origin_params, after_params)]


[tensor([[[[True, True, True],
           [True, True, True],
           [True, True, True]]]]),
 tensor([True]),
 tensor([[[[True]]]]),
 tensor([True]),
 tensor([[[[True]]]]),
 tensor([True])]

([Parameter containing:
  tensor([[[[0.1054, 0.5890, 0.5834],
            [0.1104, 0.3065, 0.0310],
            [0.1167, 0.0346, 0.3317]]]], requires_grad=True),
  Parameter containing:
  tensor([-0.3986], requires_grad=True),
  Parameter containing:
  tensor([[[[-0.0297]]]], requires_grad=True),
  Parameter containing:
  tensor([0.0152], requires_grad=True),
  Parameter containing:
  tensor([[[[0.2699]]]], requires_grad=True),
  Parameter containing:
  tensor([0.4905], requires_grad=True)],
 [Parameter containing:
  tensor([[[[0.1054, 0.5890, 0.5834],
            [0.1104, 0.3065, 0.0310],
            [0.1167, 0.0346, 0.3317]]]], requires_grad=True),
  Parameter containing:
  tensor([-0.3986], requires_grad=True),
  Parameter containing:
  tensor([[[[-0.0297]]]], requires_grad=True),
  Parameter containing:
  tensor([0.0152], requires_grad=True),
  Parameter containing:
  tensor([[[[0.2699]]]], requires_grad=True),
  Parameter containing:
  tensor([0.4905], requires_grad=True)])

In [7]:
from ganet_head import GANetHeadFast

In [29]:
head = model['head'].copy()
head.pop('type')
net = GANetHeadFast(**head)
head_name = 'keypts_head'
assert hasattr(net, head_name)
head = getattr(net, head_name)    # e.g. self.keypts_head: CtnetHead

In [23]:
head

CtnetHead(
  (hm): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [31]:
for head_name in head.heads:
    print(head_name)
    for params in getattr(head, head_name).parameters():
        params.requires_grad = False

hm


In [32]:
for head_name in head.heads:
    print(head_name)
    for params in getattr(head, head_name).parameters():
        print(params.requires_grad)

hm
False
False
False
False


In [5]:
conv = torch.nn.Conv2d(1,1,3)
def f(p):
    p.requires_grad = False
    return p
[p for p in conv.parameters()]

[Parameter containing:
 tensor([[[[ 0.2660,  0.0716,  0.2645],
           [-0.2987, -0.0760,  0.2019],
           [-0.3069, -0.2566,  0.2217]]]], requires_grad=True),
 Parameter containing:
 tensor([0.1298], requires_grad=True)]

In [6]:
!nvidia-smi

Mon Dec  5 20:50:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 67%   65C    P2   184W / 250W |   3230MiB / 11264MiB |     57%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:82:00.0 Off |                  N/A |
| 83%   

In [4]:
P, R, F = (1,2,3)
overall_result_str = ('Overall Precision: %f Recall: %f F1: %f' % (P, R, F))
overall_result_str

'Overall Precision: 1.000000 Recall: 2.000000 F1: 3.000000'

In [1]:
from shapely.geometry import Polygon, Point, LineString, MultiLineString

In [7]:
for i in MultiLineString([[(1,2),(2,3)]]).geoms:
    print(i)

LINESTRING (1 2, 2 3)


In [8]:
class A:
    def __init__(self):
        self._epoch = 0
    @property
    def epoch(self):
        return self._epoch
    def loop(self):
        self._epoch += 1
        print(self.epoch)
a = A()
a.epoch

0

In [12]:
{k:v for k,v in model['backbone']}

ValueError: too many values to unpack (expected 2)

In [5]:
model['backbone']

{'type': 'ResNet',
 'depth': 18,
 'strides': (1, 2, 2, 2),
 'num_stages': 4,
 'out_indices': (0, 1, 2, 3),
 'frozen_stages': 1,
 'norm_cfg': {'type': 'BN', 'requires_grad': True},
 'norm_eval': True,
 'style': 'pytorch'}

In [6]:
model['neck']

{'type': 'DeformFPN',
 'in_channels': [128, 256, 512],
 'out_channels': 64,
 'dcn_point_num': [7, 5, 3],
 'deconv_layer': [True, False, False],
 'deconv_before': False,
 'trans_idx': -1,
 'dcn_only_cls': True,
 'trans_cfg': {'in_dim': 512,
  'attn_in_dims': [512, 64],
  'attn_out_dims': [64, 64],
  'strides': [1, 1],
  'ratios': [4, 4],
  'pos_shape': (1, 10, 25)}}

In [7]:
model['head']

{'type': 'GANetHeadFast',
 'heads': {'hm': 1},
 'in_channels': 64,
 'branch_in_channels': 64,
 'num_classes': 1,
 'hm_idx': 0,
 'joint_nums': 1}

In [23]:
img = torch.randn((1,3,320,800))
backbone = ResNet(**get(model['backbone']))
fea = backbone(img)

In [3]:
import torch

NameError: name '_C' is not defined